In [1]:
from pathlib import Path
import pandas as pd
import supy as sp
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'docs/proc_var_info'))
    print(os.getcwd())
except:
    pass
%load_ext autoreload
%autoreload 2

In [2]:
from gen_df_state_csv import (gen_df_state, list_table, set_initcond,
                              set_runcontrol, set_input_runcontrol, gen_df_dim)
from gen_df_forcing_output_csv import gen_df_forcing, gen_df_output

# generate option rst files

## generate dataframes for variable groups

In [17]:
df_state = gen_df_state(
    list_table, set_initcond, set_runcontrol, set_input_runcontrol)
df_state.to_csv('df_state.csv')

In [16]:
%load_ext snakeviz
%snakeviz gen_df_state(list_table, set_initcond, set_runcontrol, set_input_runcontrol)

 
*** Profile stats marshalled to file '/var/folders/4d/jj1kyw857kz3rrjnzsrt7cww0000gn/T/tmpxkq2shza'. 


In [4]:
df_forcing = gen_df_forcing('SSss_YYYY_data_tt.csv')
df_forcing.to_csv('df_forcing.csv')

In [5]:
df_output = gen_df_output(
    [
        'SSss_YYYY_SUEWS_TT.csv',
        'SSss_DailyState.csv',
        'SSss_YYYY_snow_TT.csv',
    ],
)
df_output.to_csv('df_output.csv')

## generate option string for rst option file

In [6]:
def gen_opt_str(ser_rec:pd.Series)->str:
    '''generate rst option string
    
    Parameters
    ----------
    ser_rec : pd.Series
        record for specifications
    
    Returns
    -------
    str
        rst string
    '''

    name = ser_rec.name
    indent=r'    '
    str_opt = f'.. option:: {name}'+'\n\n'
    for spec in ser_rec.sort_index().index:
        str_opt += indent+f':{spec}:'+'\n'
        spec_content = ser_rec[spec]
        str_opt += indent+indent+f'{spec_content}'+'\n'
    return str_opt


# xx=df_var_info.set_index('variable').iloc[10]
# print(gen_opt_str(xx))



In [11]:
def gen_rst(path_rst, path_df_csv, rst_title):
    df_var_info = pd.read_csv(path_df_csv).set_index('variable')
    df_var_info['rst'] = df_var_info.copy().apply(gen_opt_str, axis=1)
    df_var_info = df_var_info.sort_index().reset_index(drop=True)
    rst_txt_x = '\n\n'.join(df_var_info.rst)
    rst_txt = '\n'.join([rst_title, rst_txt_x])
    with open(path_rst, 'w') as f:
        print(rst_txt, file=f)

    return path_rst


# gen_rst(
#     '../source/data-structure/test.rst', 
#     df_state,
#     'xx\n')


In [12]:
path_rst_base = Path('../source/data-structure/')


def gen_group_dict(group):
    rst_title = f'''
.. _df_{group}_options:

`df_{group}` variables
============================

'''

    dict_group = {
        'path_rst': path_rst_base/('df_'+group+'.rst'),
        'path_df_csv': 'df_'+group+'.csv',
        'rst_title': rst_title,
        }

    return dict_group


print(gen_group_dict('state'))


{'path_rst': PosixPath('../source/data-structure/df_state.rst'), 'path_df_csv': 'df_state.csv', 'rst_title': '\n.. _df_state_options:\n\n`df_state` variables\n============================\n\n'}


In [13]:
dict_rst_out = {group:gen_group_dict(group)
                for group in ['state', 'forcing', 'output']}
# dict_rst_out


In [15]:
for group in dict_rst_out:
    print(gen_rst(**dict_rst_out[group]))



../source/data-structure/df_state.rst
../source/data-structure/df_forcing.rst
../source/data-structure/df_output.rst
